In [65]:
import pandas as pd
import numpy as np

# Importing the necessary packages
import numpy as np                                  # "Scientific computing"

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation

import seaborn as sns                               # Advanced data visualisation

from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error


In [66]:
df = pd.read_csv('../../full_list_companies.csv', delimiter=';')
df.columns

Index(['BedrijfID', 'Naam', 'Locatie', 'gescrapeteData', 'AantalWerknemers',
       'Omzet', 'Balanstotaal', 'SoortBusiness', 'Framework', 'sectorID',
       'Adres'],
      dtype='object')

In [67]:
df = df[['sectorID', 'Omzet']]
df.head()

,sectorID,Omzet
0,4,0
1,7,0
2,2,0
3,2,10054586
4,2,0


In [68]:
df['Personeel'] = np.floor(np.random.normal(loc=125, scale=75, size=len(df)))

df['Omzet'] = np.floor(np.random.normal(loc=450000, scale=200000, size=len(df)))

df['Oprichtingsjaar'] = np.floor(np.random.normal(loc=2000, scale=15, size=len(df)))

randomBeursNotatie = np.random.randint(0, 2, size=len(df))

randomUrban = np.random.randint(0, 2, size=len(df))
df['Urban'] = randomUrban

df['Score'] = np.floor(np.random.normal(loc=50, scale=25, size=len(df)))

In [69]:
df.head()

df.describe()

,sectorID,Omzet,Personeel,Oprichtingsjaar,Urban,Score
count,84970.000000,8.497000e+04,84970.000000,84970.000000,84970.000000,84970.000000
mean,3.524656,4.499781e+05,124.732835,1999.448723,0.500812,49.582464
std,2.298011,1.997235e+05,74.756931,14.998096,0.500002,24.904249
min,1.000000,-3.548720e+05,-190.000000,1939.000000,0.000000,-58.000000
25%,2.000000,3.160428e+05,74.000000,1989.000000,0.000000,33.000000
50%,3.000000,4.496355e+05,125.000000,1999.000000,1.000000,50.000000
75%,5.000000,5.848220e+05,175.000000,2010.000000,1.000000,66.000000
max,9.000000,1.263129e+06,456.000000,2063.000000,1.000000,153.000000


In [70]:
X = df.drop(['Score'], axis=1)
y = df['Score']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

In [71]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
numerical_ix = X.select_dtypes(include=['int32','int64','float64']).columns

col_transform = ColumnTransformer(transformers=[
    ('std_scaler',StandardScaler(), numerical_ix)
    ])

In [72]:
from sklearn.linear_model import  LinearRegression
lin_reg = LinearRegression()

In [73]:
pipeline = Pipeline([
    ('prep', col_transform),
    ('lin_reg', lin_reg)
    ])

In [74]:
pipeline.fit(X_train,y_train)

from sklearn.model_selection import cross_val_score
-np.mean(cross_val_score(pipeline,X_train,y_train,scoring='neg_mean_squared_error',cv=5))

619.1448233127124

In [75]:
y_predict = pipeline.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test,y_predict,squared=False)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test,y_predict)

print(f'MSE: {mse}')
print(f'MAE: {mae}')

MSE: 24.974705003497117
MAE: 19.923795659909217


# Loops

In [76]:
from sklearn.linear_model import Ridge, SGDClassifier
from sklearn.ensemble import RandomForestRegressor

lin_reg = LinearRegression()
ridge_reg = Ridge(random_state=42)
sgd_reg = SGDClassifier(random_state=42)
forest_reg = RandomForestRegressor(random_state=42)

all_reg = {
  "lin_reg  ": lin_reg,
  "ridge_reg": ridge_reg, 
  "sgd_reg  ": sgd_reg,
  "rfr_reg": forest_reg
  }

In [77]:
for key, model in all_reg.items():
  pipeline = Pipeline(
    [
      ("coltr", col_transform), 
      (key, model)
      ])
      
  pipeline.fit(X_train, y_train)
  
  y_pred = pipeline.predict(X_test)
  mae = cross_val_score(pipeline, X_train, y_train, cv=3, scoring="neg_mean_squared_error").mean()
  mse = mean_absolute_error(y_test, y_pred)

  print(f"{key}\tMAE-cross validation = {-mae}\tMAE-test set = {mse}")

lin_reg  	MAE-cross validation = 619.1463662994532	MAE-test set = 19.923795659909217
ridge_reg	MAE-cross validation = 619.1463640916594	MAE-test set = 19.923795651168906


C:\Users\dylan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


sgd_reg  	MAE-cross validation = 1052.618404103553	MAE-test set = 29.90486277832698
rfr_reg	MAE-cross validation = 664.2459393838452	MAE-test set = 20.509735913006637


In [78]:
from sklearn.preprocessing import PolynomialFeatures

degrees = [i for i in range(1, 10)]
lin_reg = LinearRegression()

In [79]:
for i in degrees:
  poly = PolynomialFeatures(degree=i)
  pipeline = Pipeline(
    [
        ("coltr", col_transform),
        ("poly", poly),
        ("lin_reg", lin_reg)
        ])

  X_train_poly = pipeline.fit(X_train, y_train)

  y_pred = pipeline.predict(X_test)

  mae = cross_val_score(pipeline, X_train, y_train, cv=3, scoring="neg_mean_squared_error").mean()
  mse = mean_absolute_error(y_test, y_pred)

  print(f"degree = {i}\tMAE-cross validation = {-mae}\tMAE-test set = {mse}")

degree = 1	MAE-cross validation = 619.1463662994532	MAE-test set = 19.923795659909217
degree = 2	MAE-cross validation = 619.1463082093189	MAE-test set = 19.92726653069604
degree = 3	MAE-cross validation = 619.6710484744771	MAE-test set = 19.928711990074873
degree = 4	MAE-cross validation = 620.6806987571471	MAE-test set = 19.936403536445276
degree = 5	MAE-cross validation = 621.6708531284376	MAE-test set = 19.96167624073615
degree = 6	MAE-cross validation = 624.6038074383189	MAE-test set = 20.00059144249049
degree = 7	MAE-cross validation = 630.2321264586831	MAE-test set = 20.080460372911876
degree = 8	MAE-cross validation = 656.9146483515025	MAE-test set = 20.18636915324943
degree = 9	MAE-cross validation = 735.1502587889723	MAE-test set = 20.311508746241543
